In [10]:
!pip install torchsummary

In [37]:
import os
import sys
import torch
import torch.nn as nn
sys.path.append('../')
import config
import pickle
import numpy as np
import cal_train

from torch.nn import Conv1d, Conv2d
from torchsummary import summary
from utils import get_int_label
from Cal_Dataset import Cal_Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from torch.nn import CrossEntropyLoss
from torch.optim import Adam

In [12]:
import torch
import torch.nn as nn
sys.path.append('../')
import config

class CalNet(nn.Module):
    def __init__ (self, input_shape, n_classes):
        super(CalNet, self).__init__()
        self.net = nn.Sequential(
            # 2D Conv acts as 1D Conv
            # The correct order is: Conv > Normalization > Activation > Dropout > Pooling.
            nn.Conv2d(in_channels = 1, out_channels = 1, kernel_size = (input_shape[0], 1), padding = (0, 0)),
            nn.BatchNorm1d(1),
            nn.ReLU(),
            nn.Dropout(p = 0.3),
            nn.Linear(input_shape[1], n_classes),
            # Softmax: will be added in loss
        )
        
    def forward (self, x):
        x = self.net(x)
        return x

In [30]:
device = config.config_dict['device']
model = CalNet(input_shape = (5, 91), n_classes = 91)
model.to(device)
# summary(model, input_size = torch.Tensor(np.array((1, 5, 91))).float().to(device))
rand = np.random.rand(1, 5, 91)
rand = torch.Tensor(rand).float().to(device)
logits = model(rand)
logits = logits.detach().cpu().numpy()
prob = logits[0][0][np.argmax(logits, axis = 2)]
print(logits.shape, prob)

(1, 1, 91) [[0.8510561]]


In [44]:
cal_model = CalNet(input_shape = (5, 91), n_classes = 91)
probs, labels = cal_train.load_probs('1.0')
int_labels = np.array([get_int_label(label) for label in labels])
probs = np.array(probs)

train_probs, val_probs, train_labels, val_labels = train_test_split(probs, int_labels, test_size = 0.3, random_state = 42, shuffle = True, stratify = int_labels)
cal_train_dataset = Cal_Dataset(train_probs, train_labels)
cal_val_dataset = Cal_Dataset(val_probs, val_labels)
cal_train_loader = DataLoader(cal_train_dataset)
cal_val_loader = DataLoader(cal_val_dataset)

print(len(probs), len(int_labels), len(cal_train_dataset), len(cal_val_dataset))

Loading classification probabilities and labels from pickle file for fold 1.0
2204 2204 1542 662


In [43]:
# Define training and validating hyperparams
cfg = config.config_dict
cal_criterion = CrossEntropyLoss(weight = None)
cal_optimizer = Adam(cal_model.parameters(), lr = cfg['cal_lr'], weight_decay = cfg['weight_decay'])

In [46]:
accs = cal_train.cal_run('1.0', cal_train_loader, cal_val_loader, cal_model, cal_criterion, cal_optimizer, cfg)

  0%|          | 0/1542 [00:00<?, ?it/s]

Epoch 1/40 of the inner folds corresponding to  outer fold 1.0


RuntimeError: Expected target size [1, 91], got [1]